# OpenFold Colab

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import py3Dmol
from proteome import protein
from proteome.models.folding.openfold import config
from proteome.models.folding.openfold.modeling import OpenFoldForFolding, OPENFOLD_MODEL_URLS
from proteome.models.folding.openfold.np.relax import relax

In [ ]:
sequence = 'MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH'

In [ ]:
OPENFOLD_MODEL_URLS.keys()

In [ ]:
folder = OpenFoldForFolding(model_name="finetuning_no_templ_ptm-1", random_seed=0)

In [ ]:
#folder = OpenFoldForFolding(random_seed=0)
predicted_protein, plddt = folder.fold(sequence)

In [ ]:
unrelaxed_pdb_str = protein.to_pdb(predicted_protein)

In [ ]:
amber_relaxer = relax.AmberRelaxation(
    max_iterations=0,
    tolerance=2.39,
    stiffness=10.0,
    exclude_residues=[],
    max_outer_iterations=20,
    use_gpu=False,
)
relaxed_pdb, _, _ = amber_relaxer.process(
    prot=predicted_protein, cif_output=False
)

In [ ]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(relaxed_pdb)
color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()